In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import math
from scipy.optimize import minimize
from scipy.spatial import ConvexHull
from scipy.linalg import norm
from scipy.optimize import minimize_scalar
import datetime


data = pd.read_csv("banana.csv", sep=',')

In [28]:
def weak_learner(X, y, w):
    # Define and train the weak learner
    clf = DecisionTreeClassifier(max_depth=2)
    clf.fit(X, y,sample_weight=w)
    return clf

In [29]:
def encode_hypothesis(hypothesis, num_hypotheses):
    encoded_vector = np.zeros(num_hypotheses)
    encoded_vector[hypothesis] = 1
    return encoded_vector

In [30]:
def delta(d):
    m = len(d)
    return np.sum(d * np.log(d)) + np.log(m)

def expression(d):
    return np.dot(d.T, np.dot(A2, w)) + (1/N) * delta(d)

def minimize_expression(A2, w_1, N, v):
    m = A2.shape[0]  # Numero di righe nella matrice A
    P_m_v = np.random.rand(m) / v  # Generazione casuale di d in [0, 1/v]^m (ad esempio)
    best_d = None
    min_value = float('inf')
    for _ in range(m):  # Prova vari valori casuali di d
        d = P_m_v / np.sum(P_m_v)  # Normalizza per rispettare Σₖ dₖ¹ = 1
        value = expression(d)
        
        if value < min_value:
            min_value = value
            best_d = d

    return d, best_d

In [31]:
def short_step_FW(A2, ej2, w, result):
    lamb = []
    value_0 = np.dot(A2, (ej2 - w))
    value_1 = np.dot(result.T, value_0)
    value_2 = np.dot(A2, (ej2 - w))
    value_3 = (norm(value_2, ord=np.inf) ** 2) * N
    value = value_1 / value_3
    lamb.insert(0, np.clip(value, 0, 1))
    print(lamb)
    w_1= []
    r = (ej2 - w)
    w_1 = w + (lamb*r)
    return w_1

In [49]:
def pairwise_FW(A, d, E_t):
    best_e = None
    E_t = E_t_plus_one[:-1]
    for e in E_t:
        value = expression_f(A, d, e)
        if value < best_e:
            best_e = value
    return best_e
def expression_f(A, d, e):
    return np.dot(d.T, np.dot(A, e))
def step_size(s):
    value_0 = (ej_2 - e_away)
    value_1 = s(value_0)
    value_2 = w + value_1
    value_3 = np.dot(-A, value_2)
    value = value_3 + (1/N)* delta(result)

In [32]:
def secondary_weight(A, d, w):
    return np.dot(d.T, np.dot(A, w))

def minimize_secondary_weight(A, w):
    m = A.shape[0]
    best_d = None
    min_value = float('inf')

    for _ in range(m):
        value = secondary_weight(A2, d, w)
        if value < min_value:
            min_value = value
            best_d = d

    return best_d, min_value

def maximize_w_in_convex_hull(E_t_plus_1, A2, w):
    max_value = -float('inf')
    max_w = None
    for w_candidate in E_t_plus_1:
        d, value = minimize_secondary_weight(A2, w_candidate)
        if value > max_value:
            max_value = value
            max_w = w_candidate
    return max_w, max_value



In [33]:
def f(w):
    return np.dot(d.T, np.dot(A2, w))
def update_weights(f_w):
    best_w = None
    min_value = float('inf')
    s = A2.shape[1]
    for c in range(len(f_w)):
        for _ in range(s):
            value = f(f_w[c])
            if value < min_value:
                min_value = value
                best_w = f_w[c]
    return best_w, min_value

In [34]:
#Normalizing the variables to values in [-1,1]
for col in data.columns:
    if col != 'Class':
        min_val = data[col].min()
        max_val = data[col].max()
        data[col] = (data[col] - min_val) / (max_val - min_val)  # Normalize to range 0-1
        data[col] = data[col] * 2 - 1  # Rescale to range -1 to 1

In [35]:
#Data preparation 
train_data, test_data = train_test_split(data, test_size=0.2, random_state=123)
X_train = train_data.drop('Class', axis=1)
y_train = train_data['Class']

X_test = test_data.drop('Class', axis=1)
y_test = test_data['Class']

In [36]:
X_train

,At1,At2
3887,0.511864,0.301075
2852,0.320339,0.290323
3519,0.545763,-0.878136
4798,-0.474576,-0.373118
3176,-0.081695,0.362007
...,...,...
5218,0.327458,0.365591
4060,-0.056271,0.673835
1346,0.250169,0.030108
3454,-0.235593,0.097133


In [37]:
# create the decision tree
tree = DecisionTreeClassifier(max_depth=2)
tree.fit(X_train, y_train)

# obtain the hypothesis space hj_0
regole_matrice = tree.tree_.threshold.reshape(-1, 1)

# Stampa la matrice di regole di decisione
print("Matrice di regole di decisione:")
print(regole_matrice)

Matrice di regole di decisione:
[[ 0.07150538]
 [-0.44767025]
 [-2.        ]
 [-2.        ]
 [ 0.5440678 ]
 [-2.        ]
 [-2.        ]]


In [38]:
X_train_array = X_train.values
n_righe = X_train.shape[0]

# Numero di regole
n_regole = len(regole_matrice)

# Matrice per memorizzare le previsioni per ogni riga e regola
previsioni_totali = np.zeros((n_righe, n_regole))

# Calcola le previsioni per ciascuna riga e ciascuna regola
for i in range(n_regole):
    previsioni_totali[:, i] = y_train[i] * (weak_classifier.predict(X_train_array * regole_matrice[i]))

print("Previsioni totali:")
print(previsioni_totali)


Previsioni totali:
[[-1.  1.  1. ... -1. -1.  1.]
 [-1.  1.  1. ... -1. -1.  1.]
 [-1. -1.  1. ... -1. -1.  1.]
 ...
 [-1.  1. -1. ...  1.  1. -1.]
 [-1.  1. -1. ...  1.  1. -1.]
 [-1. -1.  1. ...  1. -1.  1.]]


C:\Users\matteo posenato\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\matteo posenato\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\matteo posenato\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\matteo posenato\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\matteo posenato\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\m

In [39]:
previsioni_totali = pd.DataFrame(previsioni_totali)
A2 = previsioni_totali
print(A2)

        0    1    2    3    4    5    6
0    -1.0  1.0  1.0 -1.0 -1.0 -1.0  1.0
1    -1.0  1.0  1.0 -1.0 -1.0 -1.0  1.0
2    -1.0 -1.0  1.0 -1.0 -1.0 -1.0  1.0
3    -1.0 -1.0 -1.0  1.0  1.0  1.0 -1.0
4    -1.0  1.0  1.0 -1.0 -1.0 -1.0  1.0
...   ...  ...  ...  ...  ...  ...  ...
4235 -1.0  1.0  1.0 -1.0 -1.0 -1.0  1.0
4236 -1.0  1.0  1.0 -1.0 -1.0 -1.0  1.0
4237 -1.0  1.0 -1.0  1.0  1.0  1.0 -1.0
4238 -1.0  1.0 -1.0  1.0  1.0  1.0 -1.0
4239 -1.0 -1.0  1.0 -1.0  1.0 -1.0  1.0

[4240 rows x 7 columns]


In [40]:
#Train the weak learner : decision tree of max depth 2 
weak_classifier = weak_learner(X_train, y_train,1)
y_pred_weak = weak_classifier.predict(X_test)
print("Predictions:", y_pred_weak)

Predictions: [-1 -1 -1 ... -1  1 -1]


[[ 0.07150538]
 [-0.44767025]
 [-2.        ]
 [-2.        ]
 [ 0.5440678 ]
 [-2.        ]
 [-2.        ]]


In [13]:
def give_the_max_edge_h(h):
    max_error = -1  # Inizializza l'errore massimo
    max_error_hypothesis = None

    for i, hypothesis in enumerate(h):
        for row in A2:
    # Calcola l'errore di classificazione per la riga corrente
            error = np.sum(np.abs(row - hypothesis))
            if error > max_error:
                max_error = error
                max_error_hypothesis = hypothesis
                max_error_hypothesis_index = i
    return max_error_hypothesis_index


In [41]:

print("Max-edge hypothesis:", max_error_hypothesis)
print("Errore massimo:", max_error)
print(max_error_hypothesis_index)

NameError: name 'max_error_hypothesis' is not defined

In [43]:
hj_1_index = np.argmax(regole_matrice)  # Find the index of the predicted hypothesis
num_hypotheses = len(regole_matrice) # Total number of hypotheses in the set H
ej1 = encode_hypothesis(hj_1_index, num_hypotheses)

In [44]:
m = A2.shape[0]

In [45]:
m

4240

In [46]:
w = np.zeros(m,)
w=ej1

In [25]:
w

array([0.       , 0.       , 0.       , 0.       , 0.5440678, 0.       ,
       0.       ])

In [40]:
v = 100

d, result = minimize_expression(A2, w, n, v)

In [41]:
#obtain the hypotesis hj t+1 
h2= weak_learner(X_train,y_train,result)
hj_2 = h2.tree_.threshold.reshape(-1, 1)

In [42]:
hj_2_index = np.argmax(hj_2)  # Find the index of the predicted hypothesis
num_hypotheses = len(hj_2)  # Total number of hypotheses in the set H

ej2 = encode_hypothesis(hj_2_index, num_hypotheses, hj_2)

In [43]:
print(hj_1)
print(hj_2)
print(ej1)
print(ej2)

[[ 0.07150538]
 [-0.44767025]
 [-2.        ]
 [-2.        ]
 [ 0.5440678 ]
 [-2.        ]
 [-2.        ]]
[[ 0.08297491]
 [-0.44713262]
 [-2.        ]
 [-2.        ]
 [ 0.49322033]
 [-2.        ]
 [-2.        ]]
[0.        0.        0.        0.        0.5440678 0.        0.       ]
[0.         0.         0.         0.         0.49322033 0.
 0.        ]


In [44]:
E_t_plus_1 = [ej1, ej2]

In [53]:
print(result)

[4.01733560e-04 2.29861394e-04 3.32219727e-04 ... 5.25071875e-05
 9.86787832e-05 3.94788196e-04]


In [61]:
min_value= -1
min_index = None
print(type(A2))
val_1 = np.min([np.dot(result,A2) for tau in range(t)])
val_2 = expression(d)
print(val_1)


<class 'pandas.core.frame.DataFrame'>


IndexError: index 7 is out of bounds for axis 0 with size 7

In [ ]:
#compute the FW weight
lamb= []
value_0 = np.dot(A2, (ej2-w))
print("value0 is: {}".format(value_0))
value_1 = np.dot(result.T, value_0)
print("value1 is: {}".format(value_1))
value_2 = np.dot(A2, (ej2 - w))
print("value2 is: {}".format(value_2))
value_3 = norm(value_2, ord=np.inf)
print("value3 is: {}".format(value_3))
value = value_1 / value_3
print("value is: {}".format(value))
lamb.insert(0, np.clip(value, 0, 1))
print("lamb is: {}".format(lamb))
w_1= []
r = (ej2 - w)
print("r is: {}".format(r))
w_1 = w + (lamb*r)
print("w_1 is: {}".format(w_1))

In [ ]:
result_w, result_value = maximize_w_in_convex_hull(E_t_plus_1, A2, w)
print("Vettore w ottimale:", result_w)
print("Valore massimo:", result_value)

In [ ]:
w_2 = result_w
f_w = []
f_w.insert(0, w_1)
f_w.insert(1, w_2)
f_w = [array.tolist() for array in f_w]

In [ ]:
best_w, v = update_weights(f_w)

In [ ]:
best_w

In [48]:
#shortstep FW
#set parameters
v = 424
T = 100
e = 0.01
epsilon = 100
N = (2 * np.log(m/v))/e
#create all the variables that we need
E_t = []
E_t_plus_1 = [ej1]
d_values =[]
for t in range(1, T):
    print("iteration number:{}".format(t))
    current_datetime = datetime.datetime.now()
    print(f"Iterazione {t} avviata il {current_datetime}")
    #create the distribution d_t
    d, result = minimize_expression(A2, w, N, v)
    #obtain the hypothesis h_j+1 and e_j+1
    d_values.append(result)
    h2= weak_learner(X_train,y_train,result)
    hj_2 = h2.tree_.threshold.reshape(-1, 1)# Total number of hypotheses in the set
    hj_2_index = np.argmax(hj_2)  # Find the index of the predicted hypothesis
    num_hypotheses = len(hj_2)  # Total number of hypotheses in the set H
    ej2 = encode_hypothesis(hj_2_index, num_hypotheses)
    for tau in range(t):
        val_1 = np.min(np.dot(d_values[tau], A2))
        val_2 = expression(d)
        epsilon_1 = np.abs(val_1 + val_2) 
        if epsilon_1 < epsilon:
            epsilon = epsilon_1
    print(epsilon)
    if epsilon <= (e/2):
        print("achieve the opptimality gap")
        break
    E_t_plus_1.insert(t, ej2)
    #compute the FW weight
    w_1 = short_step_FW(A2, ej2, w, result)
    print("The FW weight is: {}".format(w_1))
    #calculate secondary weigths
    result_w, result_value = maximize_w_in_convex_hull(E_t_plus_1, A2, w)
    w_2 = result_w
    print("The secondary weight is{}".format(w_2))
    f_w = []
    f_w.insert(0, w_1)
    f_w.insert(1, w_2)
    print(f_w)
    f_w = [array.tolist() for array in f_w]
    best_w, v = update_weights(f_w)
    w = best_w

iteration number:1
Iterazione 1 avviata il 2023-08-22 12:57:06.457976
0.52598653084278
[nan]
The FW weight is: [nan nan nan nan nan nan nan]


C:\Users\matteo posenato\AppData\Local\Temp\ipykernel_9100\1514477971.py:7: RuntimeWarning: invalid value encountered in scalar divide
  value = value_1 / value_3


The secondary weight is[0. 0. 0. 0. 1. 0. 0.]
[array([nan, nan, nan, nan, nan, nan, nan]), array([0., 0., 0., 0., 1., 0., 0.])]
iteration number:2
Iterazione 2 avviata il 2023-08-22 12:57:08.335643
0.5188281400064831
[nan]
The FW weight is: [nan nan nan nan nan nan nan]


C:\Users\matteo posenato\AppData\Local\Temp\ipykernel_9100\1514477971.py:7: RuntimeWarning: invalid value encountered in scalar divide
  value = value_1 / value_3


The secondary weight is[0. 0. 0. 0. 1. 0. 0.]
[array([nan, nan, nan, nan, nan, nan, nan]), array([0., 0., 0., 0., 1., 0., 0.])]
iteration number:3
Iterazione 3 avviata il 2023-08-22 12:57:10.304354
0.5126044736925102
[nan]
The FW weight is: [nan nan nan nan nan nan nan]


...

The secondary weight is[0. 0. 0. 0. 1. 0. 0.]
[array([nan, nan, nan, nan, nan, nan, nan]), array([0., 0., 0., 0., 1., 0., 0.])]
iteration number:98
Iterazione 98 avviata il 2023-08-22 13:24:55.089338
0.5005935937203412
[nan]
The FW weight is: [nan nan nan nan nan nan nan]


C:\Users\matteo posenato\AppData\Local\Temp\ipykernel_9100\1514477971.py:7: RuntimeWarning: invalid value encountered in scalar divide
  value = value_1 / value_3


The secondary weight is[0. 0. 0. 0. 1. 0. 0.]
[array([nan, nan, nan, nan, nan, nan, nan]), array([0., 0., 0., 0., 1., 0., 0.])]
iteration number:99
Iterazione 99 avviata il 2023-08-22 13:25:30.082323
0.5005935937203412
[nan]
The FW weight is: [nan nan nan nan nan nan nan]


C:\Users\matteo posenato\AppData\Local\Temp\ipykernel_9100\1514477971.py:7: RuntimeWarning: invalid value encountered in scalar divide
  value = value_1 / value_3


The secondary weight is[0. 0. 0. 0. 1. 0. 0.]
[array([nan, nan, nan, nan, nan, nan, nan]), array([0., 0., 0., 0., 1., 0., 0.])]


In [52]:
#pairwise FW
#set parameters
v = 424
T = 100
e = 0.01
epsilon = 100
N = (2 * np.log(m/v))/e
#create all the variables that we need
E_t = []
E_t_plus_1 = [ej1]
d_values =[]
for t in range(1, T):
    print("iteration number:{}".format(t))
    current_datetime = datetime.datetime.now()
    print(f"Iterazione {t} avviata il {current_datetime}")
    #create the distribution d_t
    d, result = minimize_expression(A2, w, N, v)
    #obtain the hypothesis h_j+1 and e_j+1
    d_values.append(result)
    h2= weak_learner(X_train,y_train,result)
    hj_2 = h2.tree_.threshold.reshape(-1, 1)# Total number of hypotheses in the set
    hj_2_index = np.argmax(hj_2)  # Find the index of the predicted hypothesis
    num_hypotheses = len(hj_2)  # Total number of hypotheses in the set H
    ej2 = encode_hypothesis(hj_2_index, num_hypotheses)
    for tau in range(t):
        val_1 = np.min(np.dot(d_values[tau], A2))
        val_2 = expression(d)
        epsilon_1 = np.abs(val_1 + val_2) 
        if epsilon_1 < epsilon:
            epsilon = epsilon_1
    print(epsilon)
    if epsilon <= (e/2):
        print("achieve the opptimality gap")
        break
    E_t_plus_1.insert(t, ej2)
    #compute the FW weight
    e_away = pairwise_FW(A, result, E_T_plus_one)
    a = []
    for i in range(len(e_away)):
        if e_away[i] == w[i]:
            a[i] = 1
        else:
            a[i] = 0
    #set lamb_t_max
    lamb_t_max = a
    print(lamb_t_max)
    #calculate lamb_t
    best_lamb = None 
    for s in range(lamb_t_max + 1):
        value = step_size(s)
        if value < best_lamb:
            best_lamb = value
    w_1 = w + best_lamb * (ej_2 - e_away)
    print("The FW weight is: {}".format(w_1))
    #calculate secondary weigths
    result_w, result_value = maximize_w_in_convex_hull(E_t_plus_1, A2, w)
    w_2 = result_w
    print("The secondary weight is{}".format(w_2))
    f_w = []
    f_w.insert(0, w_1)
    f_w.insert(1, w_2)
    print(f_w)
    f_w = [array.tolist() for array in f_w]
    best_w, v = update_weights(f_w)
    w = best_w

iteration number:1
Iterazione 1 avviata il 2023-08-22 15:15:52.608149
0.5224302091715749


NameError: name 'A' is not defined